<a href="https://colab.research.google.com/github/bakhita11/A-Machine-Learning-Framework-for-Predicting-Cancer-Risk-in-Type-2-Diabetes/blob/main/AI_Prediction_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set parameters
n_days = 30
intervals_per_day = 96  # 15-minute intervals in 24 hours
total_rows = n_days * intervals_per_day

# Generate timestamps
start_time = datetime(2025, 1, 1, 0, 0)
timestamps = [start_time + timedelta(minutes=15 * i) for i in range(total_rows)]

# Simulate BG data
np.random.seed(42)
bg_rl = np.random.normal(110, 15, size=total_rows)        # RL agent
bg_baseline = np.random.normal(140, 20, size=total_rows)  # Baseline

# Clamp values to plausible physiological range
bg_rl = np.clip(bg_rl, 40, 400)
bg_baseline = np.clip(bg_baseline, 40, 400)

# Feature engineering: rolling mean, difference, time of day
bg_rl_rolling = pd.Series(bg_rl).rolling(window=4, min_periods=1).mean()  # 1-hour rolling mean
bg_diff = bg_rl - bg_baseline
time_of_day = [(ts.hour + ts.minute/60) for ts in timestamps]

# Synthetic binary target: 1 if RL agent BG > 180 (hyperglycemia event), else 0
event = (bg_rl > 180).astype(int)

# Build DataFrame for ML
data_encoded = pd.DataFrame({
    'BG_RL_Agent': bg_rl,
    'BG_Baseline': bg_baseline,
    'BG_RL_1hRollingMean': bg_rl_rolling,
    'BG_Difference': bg_diff,
    'TimeOfDay': time_of_day,
    'event': event  # This is your binary target variable
})

# Optional: Save to CSV
data_encoded.to_csv('simulated_ml_data.csv', index=False)
print("ML-ready DataFrame 'data_encoded' created and saved as 'simulated_ml_data.csv'.")
print(data_encoded.head())


ML-ready DataFrame 'data_encoded' created and saved as 'simulated_ml_data.csv'.
   BG_RL_Agent  BG_Baseline  BG_RL_1hRollingMean  BG_Difference  TimeOfDay  \
0   117.450712    95.235375           117.450712      22.215337       0.00   
1   107.926035    97.585997           112.688374      10.340039       0.25   
2   119.715328   127.862696           115.030692      -8.147368       0.50   
3   132.845448   149.153732           119.484381     -16.308284       0.75   
4   106.487699    85.049903           116.743628      21.437796       1.00   

   event  
0      0  
1      0  
2      0  
3      0  
4      0  


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scipy.stats import randint

# Parameters
n_patients = 500
n_days = 30
intervals_per_day = 96  # 15-min intervals
total_rows_per_patient = n_days * intervals_per_day

start_time = datetime(2025, 1, 1, 0, 0)

all_patients = []

np.random.seed(42)

for pid in range(n_patients):
    timestamps = [start_time + timedelta(minutes=15 * i) for i in range(total_rows_per_patient)]
    bg_rl = np.random.normal(110, 15, size=total_rows_per_patient)
    bg_baseline = np.random.normal(140, 20, size=total_rows_per_patient)
    bg_rl = np.clip(bg_rl, 40, 400)
    bg_baseline = np.clip(bg_baseline, 40, 400)
    bg_rl_rolling = pd.Series(bg_rl).rolling(window=4, min_periods=1).mean().values
    bg_diff = bg_rl - bg_baseline
    time_of_day = [(ts.hour + ts.minute/60) for ts in timestamps]
    is_night = [1 if 0 <= ts.hour < 6 else 0 for ts in timestamps]
    interaction = bg_rl * bg_baseline
    bg_rl_lag1 = np.roll(bg_rl, 1)
    bg_rl_lag1[0] = bg_rl[0]

    noise = np.random.normal(0, 1, size=total_rows_per_patient)
    linear_score = (
        0.025 * bg_rl +
        -0.012 * bg_baseline +
        0.018 * bg_diff +
        0.009 * np.array(time_of_day) +
        0.02 * interaction +
        0.015 * bg_rl_lag1 +
        0.05 * np.array(is_night) +
        noise
    )
    threshold = np.percentile(linear_score, 50)
    event = (linear_score > threshold).astype(int)
    flip_idx = np.random.choice(total_rows_per_patient, size=int(0.05 * total_rows_per_patient), replace=False)
    event[flip_idx] = 1 - event[flip_idx]

    df = pd.DataFrame({
        'PatientID': pid,
        'Timestamp': timestamps,
        'BG_RL_Agent': bg_rl,
        'BG_Baseline': bg_baseline,
        'BG_RL_1hRollingMean': bg_rl_rolling,
        'BG_Difference': bg_diff,
        'TimeOfDay': time_of_day,
        'IsNight': is_night,
        'Interaction': interaction,
        'BG_RL_Lag1': bg_rl_lag1,
        'event': event
    })
    all_patients.append(df)

data_encoded = pd.concat(all_patients, ignore_index=True)

print("Total patients:", data_encoded['PatientID'].nunique())
print("Total rows:", len(data_encoded))
print("Target class distribution:\n", data_encoded['event'].value_counts())

# Split by patient
patient_ids = data_encoded['PatientID'].unique()
train_pids, test_pids = train_test_split(patient_ids, test_size=0.2, random_state=42, stratify=None)
train_data = data_encoded[data_encoded['PatientID'].isin(train_pids)]
test_data = data_encoded[data_encoded['PatientID'].isin(test_pids)]

X_train = train_data.drop(columns=['event', 'PatientID', 'Timestamp'])
y_train = train_data['event']
X_test = test_data.drop(columns=['event', 'PatientID', 'Timestamp'])
y_test = test_data['event']

# Feature Selection with RFECV (fit on training data only)
rf_for_fs = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rfecv = RFECV(estimator=rf_for_fs, step=1, cv=5, scoring='roc_auc', n_jobs=-1)
rfecv.fit(X_train, y_train)
selected_features = X_train.columns[rfecv.support_]
print(f"Optimal number of features: {rfecv.n_features_}")
print("Selected features:", list(selected_features))

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Hyperparameter Tuning
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}
rf = RandomForestClassifier(random_state=42)
rand_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=30, cv=5,
    scoring='roc_auc', n_jobs=-1, random_state=42
)
rand_search.fit(X_train_selected, y_train)
best_rf = rand_search.best_estimator_
print("Best hyperparameters:", rand_search.best_params_)

# Model Training and Evaluation
best_rf.fit(X_train_selected, y_train)
y_pred = best_rf.predict(X_test_selected)
y_proba = best_rf.predict_proba(X_test_selected)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"AUC: {auc:.3f}")
print("\nConfusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)

# Feature Importance Visualization
importances = best_rf.feature_importances_
sorted_idx = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 6))
plt.barh(selected_features[sorted_idx], importances[sorted_idx])
plt.xlabel("Feature Importance")
plt.title("Top Features from Random Forest Model")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Cross-Validation AUC on training set
cv_auc = cross_val_score(best_rf, X_train_selected, y_train, cv=5, scoring='roc_auc')
print(f"5-Fold Cross-Validated AUC (train set): {cv_auc.mean():.3f} ± {cv_auc.std():.3f}")

# Optional: Compute summary statistics for reporting
print("BG_RL_Agent mean ± std:", data_encoded['BG_RL_Agent'].mean(), "±", data_encoded['BG_RL_Agent'].std())
print("BG_Baseline mean ± std:", data_encoded['BG_Baseline'].mean(), "±", data_encoded['BG_Baseline'].std())
print("% Time <70 mg/dL (RL):", (data_encoded['BG_RL_Agent'] < 70).mean() * 100)
print("% Time <70 mg/dL (Baseline):", (data_encoded['BG_Baseline'] < 70).mean() * 100)
print("Coefficient of Variation (RL):", data_encoded['BG_RL_Agent'].std() / data_encoded['BG_RL_Agent'].mean() * 100)
print("Coefficient of Variation (Baseline):", data_encoded['BG_Baseline'].std() / data_encoded['BG_Baseline'].mean() * 100)


Total patients: 500
Total rows: 1440000
Target class distribution:
 event
1    720054
0    719946
Name: count, dtype: int64


# New Section